# TODO:
- set values before the first promotion
- some items are never promoted
- somehow set these values for the testing data
  - if item was promoted during test period, we don't have the number of orders!

In [1]:
use_colab = True
if use_colab:
  import os
  from google.colab import drive 
  drive.mount("/content/gdrive", force_remount=True)
  # Change directory for nicer imports
  %cd "/content/gdrive/My Drive/Data_Mining_Cup/05 Code/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Data_Mining_Cup/05 Code


In [2]:
from datetime import datetime, timedelta

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from typing import List, Callable

%matplotlib inline

In [3]:
dynamic_features = pd.read_pickle('./data/dynamic_features.pk')
static_features = pd.read_pickle('./data/static_features.pk')

In [4]:
def last_promotion(item, avg_sales_of_all_items):
  itemID = item.index.values[0][0]
  if itemID % 1000 == 0:
    print(itemID)

  first_day_of_data = item.index.get_level_values(1).to_list()[0]
  last_day_of_data = item.index.get_level_values(1).to_list()[-1]

  def get_avg_order_on_promo_day(promos):
    idx = pd.MultiIndex.from_product([[itemID], promos])
    orders_all = item.loc[idx]
    orders = orders_all[orders_all.order.notna()].order
    if len(orders) > 0:
      avg = orders.mean()
    else:
      avg = avg_sales_of_all_items
    return round(avg)

  def helper(start_date, end_date, default_sales=None):
    # Set values to entries between start_date and end_date
    days_between = (end_date - start_date).days
    first_day_to_set = start_date + timedelta(days=1)  # First day after last promotion

    if first_day_to_set < first_day_of_data:
      value_for_january_1st = (first_day_of_data - first_day_to_set).days + 1
      first_day_to_set = first_day_of_data
      days_since_last_promo = np.arange(value_for_january_1st, days_between+1)
      last_promo_sales = default_sales  # Avg of promotions for this item
    else:
      days_since_last_promo = np.arange(1, days_between+1)  # A list of increasing numbers, starting with 1
      last_promo_sales = item.loc[(itemID, start_date)].order  # Sales on the last promoted day

    item.loc[(itemID, first_day_to_set):(itemID, end_date), "days_since_last_promotion"] = days_since_last_promo    
    item.loc[(itemID, first_day_to_set):(itemID, end_date), "last_promoted_sales"] = last_promo_sales
    return item

  # Get list of dates when the item was promoted
  promotions = item.index[item.promoted].get_level_values(1).to_list()

  default_sales = get_avg_order_on_promo_day(promotions)

  last_promo = datetime(year=2017, month=12, day=1)  # First imaginary promotion

  if len(promotions) == 0:
    item = helper(last_promo, last_day_of_data, default_sales)
    return item

  for promo_day in promotions:
    # Loop through all promotions and set all intermediate values at once
    item = helper(last_promo, promo_day, default_sales)
    last_promo = promo_day

  # Add values for the days after the last promotion
  
  if (last_day_of_data - last_promo).days > 0:
    helper(last_promo, last_day_of_data)

  return item

In [22]:
def last_promotions(data, **kwargs):
  print("... adding last_promotions")
  data["days_since_last_promotion"] = np.nan
  data["last_promoted_sales"] = np.nan
  avg_sales_of_all_items = data[data['promoted']==True].groupby('itemID')['order'].mean().mean()
  data = data.groupby('itemID').apply(last_promotion, avg_sales_of_all_items = avg_sales_of_all_items)
  return data

In [24]:
# To test on only one item:
# df = dynamic_features[dynamic_features.index.get_level_values(0) == 2]
# tmp = last_promotions(df)

In [ ]:
# tmp.tail(50)

In [ ]:
result = last_promotions(dynamic_features)

... adding last_promotions
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [ ]:
check_item = 3

In [ ]:
result.loc[check_item].head(60)

,order,salesPrice,promoted,avg_salesPrice_nonunique,day_of_year,day_of_month,month,week_nr,day_of_week,de_holidays,salesPrice_bucket,days_since_last_promotion,last_promoted_sales
date,,,,,,,,,,,,,
2018-01-01,0.0,14.04,False,11.918371,1,1,1,1,0,False,3,31.0,90.0
2018-01-02,0.0,14.04,False,11.918371,2,2,1,1,1,False,3,32.0,90.0
2018-01-03,0.0,14.04,False,11.918371,3,3,1,1,2,False,3,33.0,90.0
2018-01-04,0.0,14.04,False,11.918371,4,4,1,1,3,False,3,34.0,90.0
2018-01-05,0.0,14.04,False,11.918371,5,5,1,1,4,False,3,35.0,90.0
2018-01-06,0.0,14.04,False,11.918371,6,6,1,1,5,False,3,36.0,90.0
2018-01-07,0.0,14.04,False,11.918371,7,7,1,1,6,False,3,37.0,90.0
2018-01-08,0.0,14.04,False,11.918371,8,8,1,2,0,False,3,38.0,90.0
2018-01-09,0.0,14.04,False,11.918371,9,9,1,2,1,False,3,39.0,90.0


In [ ]:
result.loc[check_item].tail(60)

,order,salesPrice,promoted,avg_salesPrice_nonunique,day_of_year,day_of_month,month,week_nr,day_of_week,de_holidays,salesPrice_bucket,days_since_last_promotion,last_promoted_sales
date,,,,,,,,,,,,,
2018-05-01,0.0,14.04,False,11.918371,121,1,5,18,1,False,3,82.0,90.0
2018-05-02,0.0,14.04,False,11.918371,122,2,5,18,2,False,3,83.0,90.0
2018-05-03,0.0,14.04,False,11.918371,123,3,5,18,3,False,3,84.0,90.0
2018-05-04,0.0,14.04,False,11.918371,124,4,5,18,4,False,3,85.0,90.0
2018-05-05,0.0,14.04,False,11.918371,125,5,5,18,5,False,3,86.0,90.0
2018-05-06,0.0,14.04,False,11.918371,126,6,5,18,6,False,3,87.0,90.0
2018-05-07,0.0,14.04,False,11.918371,127,7,5,19,0,False,3,88.0,90.0
2018-05-08,0.0,14.04,False,11.918371,128,8,5,19,1,False,3,89.0,90.0
2018-05-09,1.0,14.04,False,11.918371,129,9,5,19,2,False,3,90.0,90.0


In [ ]:
result[result.last_promoted_sales.isna() & (result.index.get_level_values(1) < datetime(year=2018, month=6, day=3))]

,,order,salesPrice,promoted,avg_salesPrice_nonunique,day_of_year,day_of_month,month,week_nr,day_of_week,de_holidays,salesPrice_bucket,days_since_last_promotion,last_promoted_sales
itemID,date,,,,,,,,,,,,,
